## Artificial neural network approximations of Cauchy inverse problem for linear PDEs

In [ ]:
from ANNPDE.PDE import ReverseChauchyPDE
from ANNPDE.PDE.shapes import (
    ElipseShape, 
    CircleShape, 
    LineShape
)
from ANNPDE.ANN import (
    LSTM,
    prepare_data
)
import plotly.graph_objs as go
from random import randint
from tqdm import tqdm
from torch import nn
import numpy as np
import torch

SEED = randint(1, 1000000)
torch.manual_seed(SEED)
print('Seed:', SEED)

### Reverse Cauchy Problem

In [ ]:
F_EXPR = 'E ** x1 * sin(x2) * cos(t)'
G_EXPR = ['E ** x1 * sin(x2)', 'E ** x1 * cos(x2)']
H_EXPR = 'E ** x1 * sin(x2)'
Xs_symbol, t_symbol = ['x1', 'x2'], 't'

print(
    '\n • ∂u(X, t)/∂t + Δu(X, t) = 0'
    '\n • u(X, t) = f =', F_EXPR,
    '\n • ∂u(X, t)/∂n = g =', G_EXPR,
    '\n • u(X, 0)  = h =', H_EXPR,
)

## Training parameters

In [ ]:
T_SAMPLE = 64
E_SAMPLE = 128
D_SAMPLE = 512
CENTER = np.array([0, 0])
RADIUS = 0.5
EDGE_CUT = [(0, np.pi * .5), (np.pi * 1, np.pi * 1.5)]

input_size = len(CENTER) + 1  # Input is (x1, x2, t)
hidden_layer_sizes = [10, 20, 70, 30]  # Customize your hidden layer sizes
batch_size_divider = 64
num_epochs = 100

ec_lambda = lambda x: "\n    ".join(
    [str(list(map(lambda x: round(x, 5), cut))) for cut in x]
)

print(
    '\nTraining parameters:'
    '\n • Time sample:', T_SAMPLE,
    '\n • Edge sample:', E_SAMPLE,
    '\n • Domain sample:', D_SAMPLE,
    '\n\n • Circle Center:', CENTER,
    '\n • Circle Radius:', RADIUS,
    f'\n • Circle Edge Cut(s): [\n    {ec_lambda(EDGE_CUT)}\n  ]',
    '\n\n • Hidden layer sizes:', hidden_layer_sizes,
    '\n • Batch Size Divider:',batch_size_divider, 
    '\n • Batch size :', ((E_SAMPLE + D_SAMPLE) * T_SAMPLE) // batch_size_divider,
    '=  (E_SAMPLE+D_SAMPLE)*T_SAMPLE)//batch_size_divider',
    '\n • Number of epochs:', num_epochs
)

## Time sampling

In [ ]:

time = LineShape(
    seed=SEED,
    n=T_SAMPLE,
    start_point=0,
    end_point=np.pi/2,
    cross_sample_generate=1,
    even_sample=True
)
time_sample = time.get()

time.plot(3)


## Shape sampling

In [ ]:
shape = CircleShape(
    seed=SEED,
    edge_n=E_SAMPLE,
    domain_n=D_SAMPLE,
    center=CENTER,
    radius=RADIUS,
    cross_sample_generate=1,
    edge_cuts_angle=EDGE_CUT,
    even_sample=True
)
edge_sample, domain_sample = shape.get()

shape.plot(1.3)

## Defining LSTM Model

In [ ]:

model = LSTM(input_size, hidden_layer_sizes)

criterion = nn.MSELoss()
pde = ReverseChauchyPDE(
    f_function=F_EXPR,
    g_function=G_EXPR,
    h_function=H_EXPR,
    x_symbols=Xs_symbol,
    time_symbol=t_symbol,
    criterion=criterion
)
optimizer = torch.optim.Adam(model.parameters())

domain_input = torch.from_numpy(
    prepare_data(time_sample, domain_sample)
).float()
edge_input = torch.from_numpy(
    prepare_data(time_sample, edge_sample)
).float()

domain_input = domain_input[torch.randperm(domain_input.size()[0])]
edge_input = edge_input[torch.randperm(edge_input.size()[0])]

batch_size_edge = int(edge_input.shape[0] / batch_size_divider)
batch_size_domain = int(domain_input.shape[0] / batch_size_divider)

print(
    '\nSampling Information:'
    '\n • Domain input shape:', tuple(domain_input.shape),
    '\n • Edge input shape:', tuple(edge_input.shape),
    '\n • Domain Batch size:', batch_size_domain,
    '\n • Edge Batch size:', batch_size_edge
)

## Training Loop

In [ ]:
for epoch in range(num_epochs):

    total_tr_loss = 0.0
    total_f_loss = 0.0
    total_g_loss = 0.0
    total_h_loss = 0.0

    print(f'\nEpoch {epoch+1}/{num_epochs} loop ...')
    
    for i in tqdm(range(batch_size_divider)):

        batch_domain = domain_input[i*batch_size_domain:(i+1)*batch_size_domain, :]
        batch_edge = edge_input[i*batch_size_edge:(i+1)*batch_size_edge, :]

        inputs = torch.cat((batch_domain, batch_edge), dim=0)
        inputs.requires_grad_(True)
        outputs = model(inputs)


        # Calculating the gradients ----------------------------------------------------

        gradients = torch.autograd.grad(
            outputs, inputs, grad_outputs=torch.ones_like(outputs), create_graph=True
        )[0]

        # Calculate the laplacian ------------------------------------------------------

        laplacians = torch.zeros(inputs.shape)

        # Calculate the second derivatives for each input dimension
        for k in range(inputs.shape[1]):
            # Calculate the gradient of the gradients (second derivatives)
            second_derivatives = torch.autograd.grad(
                gradients[:, k], 
                inputs, 
                grad_outputs=torch.ones_like(gradients[:, k]), 
                create_graph=True
            )[0]
            # Extract the diagonal elements corresponding to the second derivative of 
            # each input
            laplacians[:, k] = second_derivatives[:, k]

        laplacians = torch.sum(laplacians ** 2, dim=1, keepdim=True)

        # ------------------ Calculate Loss functions ----------------------------------
        # Calculate the loss on the domain using laplacian function
        tr_loss = criterion(
            laplacians + gradients[:, -1].unsqueeze(1), torch.zeros(laplacians.shape)
        )
        # Calculate the loss on the Edge
        f_loss = pde.loss(
            'f', 
            outputs[batch_domain.size(0): ], 
            inputs[batch_domain.size(0): ]
        )
        # Calculate the loss on the boundary using the normal vector
        g_loss = pde.loss(
            'g', 
            inputs[batch_domain.size(0):], 
            gradients[batch_domain.size(0):, : -1]
        )
        # Calculate the loss on the boundary at t = 0
        on_zero_input = torch.cat(
            (inputs[:, :-1], torch.zeros((inputs.shape[0], 1))),
            dim=1
        )
        on_zero_output = model(on_zero_input)
        h_loss = pde.loss('h', on_zero_output, on_zero_input)
        # ------------------------------------------------------------------------------

        # Calculate the combined loss
        combined_loss = tr_loss / inputs.size(0) + \
            f_loss / batch_edge.size(0) + \
                g_loss / batch_edge.size(0) + \
                    h_loss / inputs.size(0)
        # Backward and optimize
        combined_loss.backward()
        optimizer.step()

        total_tr_loss += tr_loss.item()
        total_f_loss += f_loss.item()
        total_g_loss += g_loss.item()
        total_h_loss += h_loss.item()
    
    print(
        'Epoch [{}/{}], TR Loss: {:.4f}, F Loss: {:.4f}, ' \
        'G Loss: {:.4f}, H Loss: {:.4f}'.format(
            epoch+1, 
            num_epochs, 
            total_tr_loss / batch_size_divider, 
            total_f_loss / batch_size_divider, 
            total_g_loss / batch_size_divider, 
            total_h_loss / batch_size_divider
        )
    ) 